<a href="https://colab.research.google.com/github/zbooster/Landmark-Image-Classification/blob/main/landmark_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 준비


## Google 드라이브에서 Resize된 데이터 가져오기
gdown과 unzip을 이용하여 압축을 풀어서 나의 colab 폴더에 둔다

In [1]:
!gdown 1FSb1ahdmNENxvZvg921R47_ZfNJ0O8va

Downloading...
From: https://drive.google.com/uc?id=1FSb1ahdmNENxvZvg921R47_ZfNJ0O8va
To: /content/resizeds.zip
100% 3.09G/3.09G [00:12<00:00, 238MB/s]


In [2]:
!unzip -qq '/content/resizeds.zip' -d './datasets'

# 학습준비

In [3]:
import torch
import os

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
BATCH_SIZE = 256
EPOCH = 30

In [4]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

transform_base = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor()])
train_dataset = ImageFolder(root='/content/datasets/resizeds/Training', transform=transform_base)
val_dataset = ImageFolder(root='/content/datasets/resizeds/Validation', transform=transform_base)

In [5]:
from torch.utils.data import DataLoader

train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=BATCH_SIZE,
                                           shuffle=True,
                                           num_workers=2)
val_loader = torch.utils.data.DataLoader(val_dataset,
                                         batch_size=BATCH_SIZE,
                                         shuffle=True,
                                         num_workers=2)

In [6]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):

  def __init__(self):

    super(Net, self).__init__()

    self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
    self.conv3 = nn.Conv2d(64, 64, 3, padding=1)

    self.fc1 = nn.Linear(4096, 512)
    self.fc2 = nn.Linear(512, 84)

  def forward(self, x):

    x = self.conv1(x)
    x = F.relu(x)
    x = self.pool(x)
    x = F.dropout(x, p=0.25, training=self.training)

    x = self.conv2(x)
    x = F.relu(x)
    x = self.pool(x)
    x = F.dropout(x, p=0.25, training=self.training)

    x = self.conv3(x)
    x = F.relu(x)
    x = self.pool(x)
    x = F.dropout(x, p=0.25, training=self.training)

    x = x.view(-1, 4096)
    x = self.fc1(x)
    x = F.relu(x)
    x = F.dropout(x, p=0.5, training=self.training)
    x = self.fc2(x)

    return F.log_softmax(x, dim=1)

In [7]:
model_base = Net().to(DEVICE)
optimizer = optim.Adam(model_base.parameters(), lr=0.001)

In [8]:
def train(model, train_loader, optimizer):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(DEVICE), target.to(DEVICE)
    optimizer.zero_grad()
    output = model(data)
    loss = F.cross_entropy(output, target)
    loss.backward()
    optimizer.step()

In [9]:
def evaluate(model, test_loader):
  model.eval()
  test_loss = 0
  correct = 0

  with torch.no_grad():
    for data, target in test_loader:
      data, target =  data.to(DEVICE), target.to(DEVICE)
      output = model(data)

      test_loss += F.cross_entropy(output, target, reduction='sum').item()

      pred = output.max(1, keepdim=True)[1]
      correct += pred.eq(target.view_as(pred)).sum().item()

  test_loss /= len(test_loader.dataset)
  test_accuracy = 100. * correct / len(test_loader.dataset)
  return test_loss, test_accuracy

In [10]:
import time
import copy

def train_baseline(model, train_loader, val_loader, optimizer, num_epochs=30):
  best_acc = 0.0
  best_model_wts = copy.deepcopy(model.state_dict())

  for epoch in range(1, num_epochs + 1):
    since = time.time()
    train(model, train_loader, optimizer)
    train_loss, train_acc = evaluate(model, train_loader)
    val_loss, val_acc = evaluate(model, val_loader)

    if val_acc > best_acc:
      best_acc = val_acc
      best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print('----------------- epoch {} ------------------'.format(epoch))
    print('train Loss: {:.4f}, Accuracy: {:.2f}%'.format(train_loss, train_acc))
    print('val Loss: {:.4f}, Accuracy: {:.2f}%'.format(val_loss, val_acc))
    print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

  model.load_state_dict(best_model_wts)
  return model

# 학습 및 저장

In [ ]:
base = train_baseline(model_base, train_loader, val_loader, optimizer, EPOCH)
torch.save(base, 'baseline.pt')

----------------- epoch 1 ------------------
train Loss: 2.7647, Accuracy: 34.78%
val Loss: 2.8902, Accuracy: 28.12%
Completed in 3m 54s
----------------- epoch 2 ------------------
train Loss: 1.2971, Accuracy: 68.16%
val Loss: 1.6799, Accuracy: 54.12%
Completed in 4m 0s
----------------- epoch 3 ------------------
train Loss: 0.8153, Accuracy: 80.36%
val Loss: 1.3218, Accuracy: 64.96%
Completed in 3m 46s
----------------- epoch 4 ------------------
train Loss: 0.4756, Accuracy: 88.54%
val Loss: 1.0017, Accuracy: 75.27%
Completed in 3m 45s
----------------- epoch 5 ------------------
train Loss: 0.3352, Accuracy: 92.43%
val Loss: 0.8064, Accuracy: 79.65%
Completed in 3m 43s
----------------- epoch 6 ------------------
train Loss: 0.2535, Accuracy: 94.42%
val Loss: 0.8334, Accuracy: 80.92%
Completed in 3m 42s
----------------- epoch 7 ------------------
train Loss: 0.1966, Accuracy: 96.13%
val Loss: 0.7034, Accuracy: 82.38%
Completed in 3m 41s


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

y_pred = []
y_true = []

# iterate over test data
for inputs, labels in val_loader:
        output = model_base(inputs) # Feed Network

        output = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
        y_pred.extend(output) # Save Prediction
        
        labels = labels.data.cpu().numpy()
        y_true.extend(labels) # Save Truth

# constant for classes
classes = val_dataset.classes

# Build confusion matrix
cf_matrix = confusion_matrix(y_true, y_pred)
df_cm = pd.DataFrame(cf_matrix/np.sum(cf_matrix) *10, index = [i for i in classes],
                     columns = [i for i in classes])
plt.figure(figsize = (12,7))
sn.heatmap(df_cm, annot=True)
plt.savefig('output.png')